##Makeover Monday |2025/W4 Current and Preferred Work Arrangement
Data sourced from the [Data Visualization Society](https://www.datavisualizationsociety.org/soti-report-2024)'s 2024 annual **State of the Industry (SOTI)** report that is a survey of approx 763 respondents of DVS members (46.5%) and non-members (52.2%) from 58 countries spanning all six populated continents. The purpose of this script it to load the data in via the published data set compiled by the [Makeover Monday](https://makeovermonday.co.uk/) team for use in the Tableau Makeover Monday Data Visualization challenge.

The subset of data we are looking for in this analysis focuses on two questions regarding where and how we work looking at the at the cross tabulation of the % of responsdents and their current working location vs. their preferred arrangement.

* Article Link: https://www.datavisualizationsociety.org/soti-report-2024
* Compiled Data (published by Makeover Monday: https://data.world/makeovermonday/2025-week-3-current-and-preferred-work-arrangements

Taken from the Article:

**Metholody**: *Knowing that where and how we work has changed dramatically in the past few years, we continue to track the situation data visualizers find themselves in and whether or not they are happy with their current work arrangement - be it on site, remote, or a mix of the two. Half (49%) of survey respondents who answered a question about employment told us they are currently working remotely; around one in 10 are on site (11%); and four in 10 are working in a hybrid way (39%). While 71% told us that they like their current work arrangement (whatever it may be), 10% would prefer more opportunities to interact regularly with colleagues in-person, and 19% would prefer less. This has remained fairly consistent since we first asked in 2022 although the number saying they'd prefer less time on site has increased slightly since last 2023.*

In [98]:
import pandas as pd
import numpy as np

In [106]:
#Sheet1
df_1 = pd.read_csv('https://query.data.world/s/w3irdopfjwqefqkvvsxjut7lrv6ggi?dws=00000')
#Sheet 2
df_2 = pd.read_csv('https://query.data.world/s/5snvsbwxqiiwxjaktssogsail2b3xj?dws=00000')

In [107]:
df_1.head()

,Category,Current Arrangement,Preferred Arrrangement
0,Onsite,11,6
1,Hybrid,39,40
2,Remote,49,54


In [101]:
df_1_column_sums = df_1.sum()
print(df_1_column_sums)

Category                  OnsiteHybridRemote
Current Arrangement                       99
Preferred Arrrangement                   100
dtype: object


In [59]:
df_1.dtypes

,0
Category,object
Current Arrangement,int64
Preferred Arrrangement,int64


In [60]:
df_2.head(9)

,Arrangement,preference,Percentage
0,Onsite,Onsite,36
1,Onsite,Hybrid,36
2,Onsite,Remote,27
3,Hybrid,Onsite,2
4,Hybrid,Hybrid,69
5,Hybrid,Remote,28
6,Remote,Onsite,2
7,Remote,Hybrid,18
8,Remote,Remote,81


In [61]:
df_2_column_sums = df_2.groupby("Arrangement")["Percentage"].sum()
print(df_2_column_sums)

Arrangement
Hybrid     99
Onsite     99
Remote    101
Name: Percentage, dtype: int64


In [62]:
df_2.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Arrangement,9,3,Onsite,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
preference,9,3,Onsite,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Percentage,9.0,NaN,NaN,NaN,33.222222,26.985078,2.0,18.0,28.0,36.0,81.0


In [63]:
df_2.dtypes

,0
Arrangement,object
preference,object
Percentage,int64


##The Shape of the Data
What does the shape of this data tell us and what are we looking at when inspecting the 2 dataframes provided from the datatables that Makeover Monday compiled from the DVS article.

**df_1: Distribution of Respondents Current vs Preffered Arrangments**
This df (df_1) contains the percentages from the 3 working arrangements (on site, hybrid and remote), and the percentages of respondants that currently have that arrangement and their preffered arrangement.

**df_2: Conditional Distribution of Respondents Current vs Preffered Arrangements**
This df (df_2) contains the percentages from the 3 working arrangements (on site, hybrid and remote), and based on their 2 working arragmenet, what the percentages are of their preferences for arrangment.

This lends itself to the idea of a sanky chart - because the Sankey Chart allows you to show the flow of data from one end of the visual space to another end. However the data at the moment, is aggregated (percentages) and the data is disconnected - aggregated Current vs Preffered and Current vs Preffered Interaction.

My plan to get the data in the "right shape" for a Sankey chart in Tableau would be to combine the data from the 2 df's into a "long format" required for flow visuzalition.  

##Cleaning up my Data
Preparing my data for a Sankey in Tableau will require me to reshape the data so that each preference combination represents a single "flow" row, with a source (current arrangement) and target (preffered arragement). As we not that N for his survey is **763** we can assign absolute counts (instead of percentages) but we can also assign a percentage just for spot check too/validation.

**Rounding up Data**
A quick note on rounding up data - we are looking at percentages, so one would argue that it makes sense to ensure that all of the percentages for each of the aggregated groups equal 100%. As we can see in the previous cells (column total) not all of the arrangements = 100 in df_1 and in the arrangements  perference tables (df_2) the percentages are not 100% aligned. We need to take this into consideration when converting the Percentage values into real "person" numbers. I will do this by rounding up the values to ensure total grouped aggregations = 100.

In the next steps I will:


*   Create my new dataframe with my source and target stages
*   Calculate the Value (number of people/respondents) for each of the rows based on the two dataframes provided by Makeover Monday. I will also keep the raw percentage as percentage field in order to validate.

NOTE: the df_2 is already shaped in a long format - so I can use the "Arrangment" field as the source and the "Preference" field as the "Target". The only missing piece is to work out the final N value for the missing %.

As a result, I will be **disregarding df_1 in the rest of this notebook**.



In [64]:
##Calculate the Absolute Values of the Participant in each Group based on on N = 763
N = 763
df_1_column_sums = df_1.select_dtypes("number").sum()
print(df_1_column_sums)

def get_current_dist_norm(df_1): #renormalize df_1 sp that it sums to 1.0
  current_dist = df_1.set_index("Category")["Current Arrangement"]
  current_dist_norm = current_dist / current_dist.sum() # renormalize (because it sums to 99, not 100)
  return current_dist_norm

def build_sankey_counts(df_2,current_dist_norm,N): #builds the sankey counts using the normalized percentages of current arrangment distribution
  df = df_2.copy()
  df["row_total_pct"] = df.groupby("Arrangement")["Percentage"].transform("sum")
  df["pct_within_arrangement"] = df["Percentage"] / df["row_total_pct"]

  #calculate the absolute counts using the normalize percentage
  df["arrangement_base_n"] = (df["Arrangement"].map(current_dist_norm) * N)
  df["count_float"] = (df["arrangement_base_n"] * df["pct_within_arrangement"])
  # df["count"] = (df["arrangement_base_n"] * df["pct_within_arrangement"]).round().astype(int) #note: COUNT is Number of people so should always be a whole number

  #use this (floor and remainder clause) when we want to fill in the gap to ensure that the total N finally matches
  df["count"] = np.floor(df["count_float"]).astype(int) # integer counts that SUM EXACTLY to N
  remainder = int(N - df["count"].sum())

  if remainder > 0:
      df["fraction"] = df["count_float"] - np.floor(df["count_float"])
      df.loc[df["fraction"].nlargest(remainder).index, "count"] += 1
      df.drop(columns="fraction", inplace=True)

  return df

current_dist_norm = get_current_dist_norm(df_1)
df_sankey = build_sankey_counts(df_2, current_dist_norm, N)

print("Check N:", df_sankey["count"].sum(),"<< --- Check against N:",N) # this should get you are close to your N as possible
print(f'Check sum:', current_dist_norm.sum())
df_sankey.head(9)

Current Arrangement        99
Preferred Arrrangement    100
dtype: int64
Check N: 763 << --- Check against N: 763
Check sum: 1.0


,Arrangement,preference,Percentage,row_total_pct,pct_within_arrangement,arrangement_base_n,count_float,count
0,Onsite,Onsite,36,99,0.363636,84.777778,30.828283,31
1,Onsite,Hybrid,36,99,0.363636,84.777778,30.828283,31
2,Onsite,Remote,27,99,0.272727,84.777778,23.121212,23
3,Hybrid,Onsite,2,99,0.020202,300.575758,6.072238,6
4,Hybrid,Hybrid,69,99,0.696970,300.575758,209.492195,210
5,Hybrid,Remote,28,99,0.282828,300.575758,85.011325,85
6,Remote,Onsite,2,101,0.019802,377.646465,7.478148,7
7,Remote,Hybrid,18,101,0.178218,377.646465,67.303330,67
8,Remote,Remote,81,101,0.801980,377.646465,302.864986,303


##Another round of Cleanup to get the N counts "just right"
After doing some analysis in Tableau on the first pass output - it was revealed that the inferred counts based on the percentages were not aligned when looking at the interactions reported by DVS (namely the 10% would prefer more time on site) even though we accounted for the missing percentage point in df_1 using the remainder method to ensure the N = count.sum()
![Image 1](images/mom_2025_w4/mom_2025_w4_offsite_before_df_1change.png) shows the distribution of calculated percentages for "Prefer More Onsite Time" before the "tweak" to the df_1.

Therefore, I will rerun the script above after adding an additional 1 count to the df_1 to the Current Arrangement where the categories = "Hybrid" and "Onsite"

In [109]:
#small change to df_1 Onsite Category Count percentage
df_1.loc[df_1["Category"].isin(["Onsite"]),"Current Arrangement"] += 1
df_1_column_sums = df_1.select_dtypes("number").sum(),
print(df_1_column_sums),
df_1.head()

(Current Arrangement       100
Preferred Arrrangement    100
dtype: int64,)


,Category,Current Arrangement,Preferred Arrrangement
0,Onsite,12,6
1,Hybrid,39,40
2,Remote,49,54


In [110]:
##now you can try rerunning the functions to get the DF again... we want to make sure that our totals are 100 and we are not deviating from the original data.
current_dist_norm = get_current_dist_norm(df_1)
df_sankey2 = build_sankey_counts(df_2, current_dist_norm, N)
print("Check N:", df_sankey2["count"].sum(),"<< --- Check against N:",N) # this should get you are close to your N as possible
print(f'Check sum:', current_dist_norm.sum())
df_sankey2.head(9)

Check N: 763 << --- Check against N: 763
Check sum: 1.0


,Arrangement,preference,Percentage,row_total_pct,pct_within_arrangement,arrangement_base_n,count_float,count
0,Onsite,Onsite,36,99,0.363636,91.56,33.294545,33
1,Onsite,Hybrid,36,99,0.363636,91.56,33.294545,33
2,Onsite,Remote,27,99,0.272727,91.56,24.970909,25
3,Hybrid,Onsite,2,99,0.020202,297.57,6.011515,6
4,Hybrid,Hybrid,69,99,0.696970,297.57,207.397273,207
5,Hybrid,Remote,28,99,0.282828,297.57,84.161212,84
6,Remote,Onsite,2,101,0.019802,373.87,7.403366,8
7,Remote,Hybrid,18,101,0.178218,373.87,66.630297,67
8,Remote,Remote,81,101,0.801980,373.87,299.836337,300


In [115]:
#No we just select the final fields we want for our Sankey chart in dataset in Tableau

flow_df = df_sankey2[["Arrangement","preference","count","count_float"]].copy()

print(f'Check N:', df_sankey2["count"].sum()),
flow_df.head(10)

Check N: 763


,Arrangement,preference,count,count_float
0,Onsite,Onsite,33,33.294545
1,Onsite,Hybrid,33,33.294545
2,Onsite,Remote,25,24.970909
3,Hybrid,Onsite,6,6.011515
4,Hybrid,Hybrid,207,207.397273
5,Hybrid,Remote,84,84.161212
6,Remote,Onsite,8,7.403366
7,Remote,Hybrid,67,66.630297
8,Remote,Remote,300,299.836337


#Save to BigQuery Table
Save the new flow_df table to dedicated tables in my BigQuery warehouse
This is my final step before pushing it to Google Sheets that I will use to connect to Tableau Public.

In [116]:
#convert the df to target table in bigquery dataet
UPLOAD_TO_BQ = True # set True when you actually want to write tables

if UPLOAD_TO_BQ:
  project_id = 'data-projects-478723' #'your_project_id'
  destination_table = 'makeover_monday.dvs_workarrangement' #'your_dataset.another_new_table'

  flow_df.to_gbq(
      destination_table=destination_table,
      project_id=project_id,
      if_exists='replace' ## 'if_exists' options: 'fail', 'replace', 'append'
  )

/tmp/ipython-input-1681259292.py:8: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  flow_df.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 9845.78it/s]


#**Tableau Public Dashboard**
Check out the final Tableau Dashboard:[Are Most Data Vizzers working where they want to?](https://public.tableau.com/app/profile/allison.jones/viz/MoM2025_w4CurrentandPrefferedWorkArrangement/AreMostDataVizzersworkingwheretheywantto) that explores the various preferences we have inferred from the percentages provided in the data set by Makeover Monday based on the subset of data collected in the State of the Industry Survery. For the most part, respondents were happy with their current arrangemetn, but for those who were not, there was a strong gravitation toward more hybrid/remote arrangement (aka less on site time) as illustrate in the powerfully strong flows for Hybrid and Remote to Hybrid and Remote.

![Are Most Data Vizzers working where they want to?](images/mom_2025_w4/tableadashbord_mom_2025_w4_workarrangements.png)